## Load Data

In [8]:
# load data
# normalize data

### Table Rebound Data

In [1]:
q_before_data_TRM = []
q_after_data_TRM = []

# v_ball_x, v_ball_y, v_ball_z, omega_ball_x, omega_ball_y, omega_ball_z
# or
# v_ball_x, v_ball_y, v_ball_z, F_ball_magnus_x, F_ball_magnus_y, F_ball_magnus_z
input_TRM = 6

# v_ball_x, v_ball_y, v_ball_z, omega_ball_x, omega_ball_y, omega_ball_z
# or
# v_ball_x, v_ball_y, v_ball_z, F_ball_magnus_x, F_ball_magnus_y, F_ball_magnus_z
output_TRM = 6

### Racket Rebound Data

In [ ]:
q_before_data_RRM = []
q_after_data_RRM = []

# v_ball_x, v_ball_y, v_ball_z, omega_ball_x, omega_ball_y, omega_ball_z, v_racket_y, v_racket_z, n_racket_x, n_racket_y, n_racket_z
# or
# v_ball_x, v_ball_y, v_ball_z, F_ball_magnus_x, F_ball_magnus_y, F_ball_magnus_z, v_racket_x, v_racket_y, v_racket_z, n_racket_x, n_racket_y, n_racket_z
input_RRM = 12

# v_ball_x, v_ball_y, v_ball_z, omega_ball_x, omega_ball_y, omega_ball_z
# or
# v_ball_x, v_ball_y, v_ball_z, F_ball_magnus_x, F_ball_magnus_y, F_ball_magnus_z
output_RRM = 6

## Gaussian Process Training

In [19]:
import os
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_dir = "/home/lis/switchdrive/80_data/table_tennis/" 

model_type = "TRM"
trainer_name = "GP"
save_dir = save_dir + current_time + "_" + model_type + "_" + trainer_name

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if model_type == "TRM":
    input_data = q_before_data_TRM
    output_data = q_after_data_TRM
elif model_type == "RRM":
    input_data = q_before_data_RRM
    output_data = q_after_data_RRM
else:
    raise ValueError(f"Model type {model_type} unknown.")

In [ ]:
from ball_prediction.contact_models.model_learning import GaussianProcessTraining
from sklearn.gaussian_process.kernels import Kernel, RBF, Matern, DotProduct, RationalQuadratic, ConstantKernel

#kernel = Matern(length_scale=1.0, nu=1.5)  # Choose a kernel
#kernel_params = {"length_scale": 2.0, "nu": 2.5}  # Set kernel parameters

kernel = RBF(length_scale=2.0)  # Choose an RBF kernel with length_scale=2.0
kernel_params = {"length_scale_bounds": (0.1, 10.0)}  # Set kernel parameters

gp_trainer = GaussianProcessTraining(
    kernel=kernel,
    kernel_params=kernel_params,
    use_tensorboard=True,
    use_wandb=True,
)

# Train and evaluate the Gaussian Process
gp_trainer.train(input_data, output_data)
mse = gp_trainer.evaluate(input_data, output_data)

print("Mean Squared Error:", mse)

# Save the trained GP and kernel
save_dir = "saved_gp"
gp_trainer.save(save_dir)

# Load the trained GP and kernel
loaded_gp_trainer = GaussianProcessTraining.load(save_dir)

## DNN Training

In [17]:
import os
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_dir = "/home/lis/switchdrive/80_data/table_tennis/" 

model_type = "TRM"
trainer_name = "DNN"
save_dir = save_dir + current_time + "_" + model_type + "_" + trainer_name

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if model_type == "TRM":
    input_data = q_before_data_TRM
    output_data = q_after_data_TRM

    input_dim = input_TRM
    output_dim = output_TRM
elif model_type == "RRM":
    input_data = q_before_data_RRM
    output_data = q_after_data_RRM

    input_dim = input_RRM
    output_dim = output_RRM
else:
    raise ValueError(f"Model type {model_type} unknown.")

### Specify model

In [ ]:
import torch.optim as optim

from ball_prediction.contact_models.model_learning import DNNTraining
from ball_prediction.contact_models.model_learning import ReboundModel

num_samples = 1000
num_epochs = 100
learning_rate = 0.001
hidden_neurons = [128, 64]  # Number of neurons for each hidden layer
use_layer_norm = True  # Enable or disable layer normalization
dropout_rate = 0.2  # Dropout rate, set to 0 to disable dropout

model = ReboundModel(
    input_dim=input_data,
    output_dim=output_data,
    hidden_neurons=hidden_neurons,
    use_layer_norm=use_layer_norm,
    dropout_rate=dropout_rate,
)

optimizer_class = optim.Adam  # Choose your optimizer class
optimizer_params = {"betas": (0.9, 0.999)}  # Specify optimizer parameters

DNN_trainer = DNNTraining(
    model=model,
    optimizer_class=optimizer_class,
    optimizer_params=optimizer_params,
    num_epochs=100,
    learning_rate=0.001,
    use_tensorboard=True,
    use_wandb=True,
)

DNN_trainer.train(input_data, output_data)
DNN_trainer.save(save_dir)

### Hyperparameter tuning

In [ ]:
from ball_prediction.contact_models.hyperparameter_tuning import (
    population_based_search,
    bayesian_optimization,
    random_search,
)
from ball_prediction.contact_models.model_learning import DNNTraining
from ball_prediction.contact_models.model_learning import ReboundModel


best_params, best_score = population_based_search(
    model_class=ReboundModel,
    training_class=DNNTraining,
    input_data=input_data,
    output_data=output_data,
    input_dim=input_dim,
    output_dim=output_dim,
)

## PINN Training


In [ ]:
import os
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_dir = "/home/lis/switchdrive/80_data/table_tennis/" 

model_type = "TRM"
trainer_name = "PINN"
save_dir = save_dir + current_time + "_" + model_type + "_" + trainer_name

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if model_type == "TRM":
    input_data = q_before_data_TRM
    output_data = q_after_data_TRM

    input_dim = input_TRM
    output_dim = output_TRM
elif model_type == "RRM":
    input_data = q_before_data_RRM
    output_data = q_after_data_RRM

    input_dim = input_RRM
    output_dim = output_RRM
else:
    raise ValueError(f"Model type {model_type} unknown.")

### Manual parameter selection

In [ ]:
import torch

def custom_physics_loss(
    predicted: torch.Tensor,
    ground_truth: torch.Tensor,
    output_pred: torch.Tensor,
) -> torch.Tensor:
    # Custom physics loss implementation
    loss = ...  # Your custom loss calculation here
    return loss

In [ ]:
from ball_prediction.contact_models.model_learning import PINNTraining
from ball_prediction.contact_models.model_learning import ReboundModel

num_samples = 1000
num_epochs = 100
learning_rate = 0.001
hidden_neurons = [128, 64]  # Number of neurons for each hidden layer
use_layer_norm = True  # Enable or disable layer normalization
dropout_rate = 0.2  # Dropout rate, set to 0 to disable dropout

model = ReboundModel(
    input_dim=input_dim,
    output_dim=output_dim,
    hidden_neurons=hidden_neurons,
    use_layer_norm=use_layer_norm,
    dropout_rate=dropout_rate,
)
optimizer_class = optim.Adam  # Choose your optimizer class
optimizer_params = {"betas": (0.9, 0.999)}  # Specify optimizer parameters

PINN_trainer = PINNTraining(
    model=model,
    optimizer_class=optimizer_class,
    optimizer_params=optimizer_params,
    num_epochs=100,
    learning_rate=0.001,
    physics_loss_fn=None,  # Provide your custom physics loss function
    use_tensorboard=True,
    use_wandb=True,
)

PINN_trainer.train(input_data, output_data)
PINN_trainer.save(save_dir)

### Hyperparameter Tuning

In [ ]:
from ball_prediction.contact_models.hyperparameter_tuning import (
    population_based_search,
    bayesian_optimization,
    random_search,
)
from ball_prediction.contact_models.model_learning import PINNTraining
from ball_prediction.contact_models.model_learning import ReboundModel


best_params, best_score = population_based_search(
    model_class=ReboundModel,
    training_class=PINNTraining,
    input_data=input_data,
    output_data=output_data,
    input_dim=input_dim,
    output_dim=output_dim,
)